# Maen - Multiple agents ecosystem network

There is a usage of Maen library in generatig a graph neural network to classify time series.



Load libraries

In [ ]:
using CDalgs, Maen
using BSON, DataFrames
using Graphs, SimpleWeightedGraphs
using Flux, StatsBase, LinearAlgebra, IterTools, Random
using Plots

Load dataset

In [ ]:
dataset_a = BSON.load("dataset_a.bson")
data_tensor = dataset_a[:data_tensor];

Correlation graph on angents with features & clustering function

In [ ]:
function get_clusters(smoothness, clustering)
    clustering(correlation_graph(data_tensor; smoothness=smoothness))
end

Evaluate clustering methods

In [ ]:
ncc95(g) = nc_clustering(g; α=.95)
ncc99(g) = nc_clustering(g; α=.99)
ncc999(g) = nc_clustering(g; α=.999)
clustering_algs = [louvain_clustering, ncc95, ncc99, ncc999, cdep_clustering]

smoothness = 0.8
clustering_alg = clustering_algs[1]

clusters = get_clusters(smoothness, clustering_alg);
while length(unique(clusters)) != 6
    clusters = get_clusters(smoothness, clustering_alg)
end

println(string("There are ", length(unique(clusters)), " clusters.\n",
    "Create network with ", length(unique(clusters)), " InputAgents and one OutputAgent. 
    The OutputAgent should return 4 scalars correspondign to the labes of material parameters."));

Visualise results of clustering on all samples from the dataset (show distribution)

In [ ]:
p = plot(xlabel="Deflection [m]", ylabel="Loads [MN]")
for i=1:100
    sample_i = data_tensor[i, :, :].*(-1)
    last_point_idx = 1
    for j in unique(clusters)
        cmap_j_idxs = findall(x->x == j, clusters)
        tmp = sample_i[union(cmap_j_idxs, last_point_idx), :]
        last_point_idx = maximum(cmap_j_idxs)
        scatter!(p, tmp[:, 1], tmp[:, 2], color= j!=0 ? palette(:tab20)[j] : :white, legend=false, markersize=2.5)
    end
    scatter!(p, 
        [sample_i[argmax(sample_i[:, 2]), :][1]], 
        [sample_i[argmax(sample_i[:, 2]), :][2]], color=:black, legend=false
    )
end
p

Visualise results of clustering on single particular sample

In [ ]:
function sample_plot(idx::Int64)
    p = plot(xlabel="Deflection [m]", ylabel="Loads [MN]", xticks=(1, ""), yticks=(1, ""))
    sample_i = data_tensor[idx, :, :].*(-1)
    last_point_idx = 1
    for j in unique(clusters)
        cmap_j_idxs = findall(x->x == j, clusters)
        tmp = sample_i[union(cmap_j_idxs, last_point_idx), :]
        last_point_idx = maximum(cmap_j_idxs)
        scatter!(p, tmp[:, 1], tmp[:, 2], color = j != 0 ? palette(:tab20)[j] : :white, legend=false, markersize=5)
    end
    plot!(p, sample_i[:, 1], sample_i[:, 2], color=:black, legend=false, ls=:dash)
    scatter!(p, 
        [sample_i[argmax(sample_i[:, 2]), :][1]], 
        [sample_i[argmax(sample_i[:, 2]), :][2]], color=:black, legend=false
    )
    return p
end

plot(sample_plot(25), sample_plot(50), sample_plot(66), sample_plot(91), layout=(2,2))


Prepare data and labels

In [ ]:
function normalise!(d)
    z(s, μ, σ, ϵ) = (s - μ) / (σ + ϵ)
    for i=1:size(d)[2]
        tmp = d[:, i]
        d[:, i] = z.(tmp, mean(tmp), std(tmp), 10e-10)
    end
    return d
end

for i=1:size(data_tensor)[2]
    data_tensor[:, i, :] = normalise!(data_tensor[:, i, :])
end

data = map(s -> 
    map(i -> 
        s[findall(x -> x == i, clusters), :], 1:6)
    , eachslice(data_tensor; dims=1)
)

labels = deepcopy(Matrix(dataset_a[:parameters]))
normalise!(labels)
labels = Vector{Float32}[eachrow(Matrix(labels))...]

shuffle_vec = shuffle(1:length(data))

function minibatch()
    #idx = sample(1:length(data[1:80]), 1000, replace=true)
    #data[1:80][idx], labels[1:80][idx]
    data[shuffle_vec[1:80]], labels[shuffle_vec[1:80]]
end

function testbatch()
    data[shuffle_vec[80:end]], labels[shuffle_vec[80:end]]
end


Create the Ecosystem from the XGML file and input ids mapping

In [ ]:
eco = create_ecosystem("network.xgml");
eco.ii = Dict(1:6 .=> 1:6);

Define models of components

In [ ]:
params_objects = []

function get_model_dense(in::Int64, out::Int64)
    d = Dense(in, out)
    function get_model_dense(x)
        reduce(vcat, d(x))
    end
    append!(params_objects, [d])
    return get_model_dense
end

for i=1:6
    eco.comps[string("h",i)].model = get_model_dense(length(filter(x->x==i, clusters)), 40)
end

for i=1:6
    eco.comps[string("hm",i)].model = get_model_dense(80, 40)
end

function get_model_output(in::Int64, out::Int64)
    d1 = Dense(in, 8)
    d2 = Dense(8, out)
    function get_model_output(x)
        d2(d1(vcat(x...)))
    end
    append!(params_objects, [d1, d2])
    return get_model_output
end

eco.comps["out1"].model = get_model_output(6*40, 4)

eco.schc = scv(eco.comps, eco.sch)
eco.ps_obj = params_objects

function nn(input_data::Any)
    reduce(vcat, Maen.model(eco, input_data)[end])
end

Training

In [ ]:
minibatchrun = s -> map(x -> nn(x), s)
loss = (x, y) -> mean(Flux.mse.(minibatchrun(x), y))

dev = (x, y) -> mean(
    abs.(reduce(hcat, minibatchrun(x)) .- reduce(hcat, y))./abs.(reduce(hcat, y)),
    dims=1
)
acc = (x, y) -> mean(dev(x, y) .<= 0.2)

Flux.Optimise.train!(
    loss, Flux.params(eco.ps_obj), 
    repeatedly(minibatch, 1000), ADAM()
)

println("total train: accuracy = ", acc(minibatch()...), " loss = ", loss(minibatch()...));
println("total test: accuracy = ", acc(testbatch()...), " loss = ", loss(testbatch()...));

sum(length, Flux.params(eco.ps_obj))
